In [7]:
import sys
import os
import shutil
import zipfile
import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

sys.path.append('/workspace/')

In [12]:
path_download = os.path.join('/workspace',"data", "zip")
os.makedirs(path_download, exist_ok = True)

path_unzip = os.path.join('/workspace',"data", "unzip")
os.makedirs(path_unzip, exist_ok = True)

In [9]:
zip_files = {
    'oisddata.zip': 'https://www.bankofengland.co.uk/-/media/boe/files/statistics/yield-curves/oisddata.zip',
    'latest-yield-curve-data.zip': 'https://www.bankofengland.co.uk/-/media/boe/files/statistics/yield-curves/latest-yield-curve-data.zip'
 }

In [14]:
def download(download_path, files_to_download):
    
    # Remove and create empty folder before download
    shutil.rmtree(download_path)
    os.makedirs(path_download, exist_ok = True)

    # Set path Selenium
    WINDOW_SIZE = "1280,720"

    # Options
    chrome_options = Options()
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

    #chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2, 
            "download.default_directory" : download_path,
            "disable-popup-blocking": "true"}
    chrome_options.add_experimental_option("prefs",prefs)
    # Get the response and print title
    driver = webdriver.Chrome(options=chrome_options)
    for file in files_to_download.keys():
        driver.get(files_to_download[file])
    
    # Check if file downloaded
    start_time = time.time()
    now = 0
    time_out = 10
    files = []
    while (sorted(files) != sorted(list(files_to_download.keys()))) and (now < time_out):
        files = os.listdir(download_path)
        now = time.time() - start_time
    
    driver.close()

    if now >= time_out:
        raise Exception('Time Out Error!')
    
def unzip(download_path, unzip_path, files_to_download):
    for file in files_to_download.keys():
        with zipfile.ZipFile(os.path.join(download_path, file), 'r') as zip_ref:
            zip_ref.extractall(os.path.join(unzip_path, file.split('.')[0]))

In [15]:
download(path_download, zip_files)
unzip(path_download, path_unzip, zip_files)